In [1]:
#Dependencies
import pandas as pd
from random import randint
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stat


In [2]:
# Reading the data for 2019
file = "resources/nyts2019.csv"
df2019 = pd.read_csv(file)
df2019.head()



,psu,StudentLoginID,Q1,Q2,Q3,Q5A,Q5B,Q5C,Q5D,Q5E,...,Q35,Q77,Q89,Q90,Q91,Q92,Q93,Q94,Q95,Q96
0,58123,25830,7,2,4,.N,.N,.N,.N,.N,...,.S,1,3,3,5,4,3,3,4,3
1,58123,25831,8,1,4,NaN,NaN,NaN,NaN,1,...,.S,1,3,1,3,5,4,1,4,3
2,58123,25832,6,1,4,1,1,NaN,NaN,NaN,...,.S,1,5,1,3,4,4,1,3,3
3,58123,25833,6,1,4,.N,.N,.N,.N,.N,...,.S,1,3,1,4,4,2,1,2,2
4,58123,25834,7,1,4,NaN,NaN,NaN,NaN,1,...,.S,1,6,6,6,6,6,6,6,6


In [3]:
# Rename columns so that they are differentiated
df_2019 = df2019.rename(columns={"Q1": "age19", "Q2": "gender", "Q3": "grade",
                                "Q5A": "race_a", "Q5B": "race_b", "Q5C": "race_c", 
                                "Q5D": "race_d", "Q5E": "race_e", "Q6": "try_cig", "Q7": "age_cig", 
                                 "Q8": "cig", "Q34": "try_vap", 
                                "Q35": "age_vap ", "Q77": "real_cost", 
                                "Q89": "internetAd_cig ", "Q90": "printAd_cig", 
                                "Q91": "storeAd_cig", "Q92": "streamAd_cig", 
                                "Q93": "internetAd_vap", "Q94": "printAd_vap", 
                                "Q95": "storeAd_vap", "Q96": "streamAd_vap"})
df_2019.head()

,psu,StudentLoginID,age19,gender,grade,race_a,race_b,race_c,race_d,race_e,...,age_vap,real_cost,internetAd_cig,printAd_cig,storeAd_cig,streamAd_cig,internetAd_vap,printAd_vap,storeAd_vap,streamAd_vap
0,58123,25830,7,2,4,.N,.N,.N,.N,.N,...,.S,1,3,3,5,4,3,3,4,3
1,58123,25831,8,1,4,NaN,NaN,NaN,NaN,1,...,.S,1,3,1,3,5,4,1,4,3
2,58123,25832,6,1,4,1,1,NaN,NaN,NaN,...,.S,1,5,1,3,4,4,1,3,3
3,58123,25833,6,1,4,.N,.N,.N,.N,.N,...,.S,1,3,1,4,4,2,1,2,2
4,58123,25834,7,1,4,NaN,NaN,NaN,NaN,1,...,.S,1,6,6,6,6,6,6,6,6


In [15]:
# Recoding Race using the given codebook information
# A- American Indian or alaskan native
# B - Asian
# C - Black or African American
# D - Native Hawaiian or Other Pacific Islander
# E - White
# Planning to make "race" a sring with 3 values- "Black", "white" and "Other". 

#Defining a function for combining the dummy race variables to one race column
def f(row):
    if row["race_a"] == "1":
        val = "Other"
    elif row["race_b"] == "1":
        val = "Other"
    elif row["race_d"] == "1":
        val = "Other"
    elif row['race_c'] == "1":
        val = "Black"
    elif row["race_e"] == "1":
        val = "White"
    else:
        val = " "
    return val
        
# Creating a new column race using the above function
df_2019['Race'] = df_2019.apply(f, axis=1)

# Viewing the dataframe
df_2019

# Deleting columns race_a through race_e
df_2019_recoded = df_2019.drop(['psu','race_a', 'race_b', 'race_c', 'race_d', 'race_e'], axis = 1)

df_2019_recoded

,StudentLoginID,age19,gender,grade,try_cig,age_cig,cig,try_vap,age_vap,real_cost,internetAd_cig,printAd_cig,storeAd_cig,streamAd_cig,internetAd_vap,printAd_vap,storeAd_vap,streamAd_vap,Race
0,25830,7,2,4,2,.S,.S,2,.S,1,3,3,5,4,3,3,4,3,
1,25831,8,1,4,2,.S,.S,2,.S,1,3,1,3,5,4,1,4,3,White
2,25832,6,1,4,2,.S,.S,2,.S,1,5,1,3,4,4,1,3,3,Other
3,25833,6,1,4,2,.S,.S,2,.S,1,3,1,4,4,2,1,2,2,
4,25834,7,1,4,2,.S,.S,2,.S,1,6,6,6,6,6,6,6,6,White
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19013,17553,5,2,3,2,.S,.S,2,.S,1,3,4,4,3,4,3,3,3,White
19014,17554,6,1,3,2,.S,.S,2,.S,1,4,1,2,3,5,1,2,3,White
19015,17558,6,2,3,2,.S,.S,1,7,1,2,1,5,3,5,1,3,3,White
19016,17559,5,1,3,2,.S,.S,2,.S,1,3,1,4,3,3,1,3,3,Other


In [ ]:
# Recoding the cigarette smoking and vaping variables to yes or no as the final dependant variables of interest


In [ ]:
# Merging 2019 and 2018 data using inner join

merge_df = pd.merge(df2019, df2018, on="StudentLoginID")
merge_df